# Does economic policy uncertainty influence the fair value of USDCNY?

In [1]:
import pandas as pd
import requests
import json
import inspect
%pip install quandl
%pip install fredapi
%pip install DBnomics
import quandl
quandl.ApiConfig.api_key = "P8E9VjuyG6eJfHve8-rU"
import datetime as dt
import time
import csv
import os

import inspect

os.chdir("C:/Users/marti/Dropbox/python_work/")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Get economic data

cpi_china = pd.DataFrame(quandl.get("NBSC/A01010101_M") - 100)
cpi_china.columns = ["cpi_china_yoy"]
cpi_china.rename_axis("date", axis='index', inplace=True)
cpi_china = cpi_china.loc["2005-01-31": "2020-06-30"]

cpi_usa = pd.DataFrame(quandl.get("FRED/SUUR0000SA0"))
cpi_usa = cpi_usa.pct_change(periods=12)*100
cpi_usa = cpi_usa.resample('M').last() ## Change date to be last day of the month
cpi_usa.columns = ["cpi_usa_yoy"]
cpi_usa = cpi_usa.loc["2005-01-31": "2020-06-30"]

# Calculate the difference between Chinese and US inflation rates
cpi_differential = pd.DataFrame()
cpi_differential["inflation_chinaminusus"] = cpi_china["cpi_china_yoy"] - cpi_usa["cpi_usa_yoy"]
cpi_differential.rename_axis("date", axis='index', inplace=True)

m2_yoy_china = pd.DataFrame(quandl.get("NBSC/A0B0102_M"))
m2_yoy_china.columns = ["m2_yoy_china"]
m2_yoy_china = m2_yoy_china.loc["2005-01-31": "2020-06-30"]

# USDCNY
usdcny = pd.DataFrame(quandl.get("FRED/EXCHUS"))
usdcny = usdcny.resample('M').last() ## Change date to be last day of the month
usdcny.columns = ["usdcny"]
usdcny.rename_axis("date", axis='index', inplace=True)
usdcny = usdcny.loc["2005-01-31": "2020-06-30"]

# US yields
usa_treasuries = pd.DataFrame(quandl.get("USTREASURY/YIELD"))
usa_treasuries = usa_treasuries.resample('M').last() ## Change date to be last day of the month
usa_treasuries.columns = usa_treasuries.columns.str.replace(' ', '').str.lower()
usa_treasuries = usa_treasuries.rename({'Date': 'date'},axis=1)
usa_treasuries = usa_treasuries.loc["2005-01-31": "2020-06-30"]

usa_10yr = pd.DataFrame()
usa_10yr["usa_10yr"] = usa_treasuries["10yr"]
usa_10yr.rename_axis("date", axis='index', inplace=True)

usa_2yr = pd.DataFrame()
usa_2yr["usa_2yr"] = usa_treasuries["2yr"]
usa_2yr.rename_axis("date", axis='index', inplace=True)

usa_yieldcurve_2vs10 = pd.DataFrame()
usa_yieldcurve_2vs10["usa_yieldcurve_2vs10"] = usa_10yr["usa_10yr"] - usa_2yr["usa_2yr"]


In [3]:
## Get economic policy uncertainty data
china_epu_english = pd.DataFrame()
china_epu_english = pd.read_excel("https://www.policyuncertainty.com/media/SCMP_China_Policy_Uncertainty_Data.xlsx")

china_epu_chinese_tpu = pd.DataFrame()
china_epu_chinese_tpu = pd.read_excel("https://www.policyuncertainty.com/media/China_Mainland_Paper_EPU.xlsx", 'TPU 2000 onwards', dtype={'year':int, 'month': int, 'TPU': int})

china_epu_chinese_epu = pd.DataFrame()
china_epu_chinese_epu = pd.read_excel("https://www.policyuncertainty.com/media/China_Mainland_Paper_EPU.xlsx", 'EPU 2000 onwards', dtype={'year':int, 'month': int, 'EPU': int})

china_epu_english.drop(china_epu_english.index[310], inplace=True)
china_epu_english["day"] = "28"
china_epu_english["date"] = pd.to_datetime(china_epu_english[['year', 'month', 'day']])
from pandas.tseries.offsets import MonthEnd
china_epu_english['date'] = pd.to_datetime(china_epu_english['date'], format="%Y%m")+MonthEnd(0)
china_epu_english.drop(['year', 'month', 'day'], axis=1, inplace=True)
china_epu_english.set_index("date", inplace=True)
china_epu_english.rename(columns={china_epu_english.columns[0]: "china_epu_english"}, inplace = True)
china_epu_english = china_epu_english.loc["2005-01-31": "2020-06-30"]

china_epu_chinese_tpu["day"] = "28"
china_epu_chinese_tpu["date"] = pd.to_datetime(china_epu_chinese_tpu[['year', 'month', 'day']])
china_epu_chinese_tpu['date'] = pd.to_datetime(china_epu_chinese_tpu['date'], format="%Y%m")+MonthEnd(0)
china_epu_chinese_tpu.drop(['year', 'month', 'day'], axis=1, inplace=True)
china_epu_chinese_tpu.set_index("date", inplace=True)
china_epu_chinese_tpu.rename(columns={china_epu_chinese_tpu.columns[0]: "china_epu_chinese_tpu"}, inplace = True)
china_epu_chinese_tpu.columns = ['china_epu_chinese_tpu', 'column2', 'column3', 'column4'] 
china_epu_chinese_tpu.drop(["column2", "column3", "column4"],axis=1, inplace=True)
china_epu_chinese_tpu = china_epu_chinese_tpu.loc["2005-01-31": "2020-06-30"]

china_epu_chinese_epu["day"] = "28"
china_epu_chinese_epu["date"] = pd.to_datetime(china_epu_chinese_epu[['year', 'month', 'day']])
china_epu_chinese_epu['date'] = pd.to_datetime(china_epu_chinese_epu['date'], format="%Y%m")+MonthEnd(0)
china_epu_chinese_epu.drop(['year', 'month', 'day'], axis=1, inplace=True)
china_epu_chinese_epu.set_index("date", inplace=True)
china_epu_chinese_epu.rename(columns={china_epu_chinese_epu.columns[0]: "china_epu_chinese_epu"}, inplace = True)
china_epu_chinese_epu.columns = ['china_epu_chinese_epu', 'column2', 'column3', 'column4'] 
china_epu_chinese_epu.drop(["column2", "column3", "column4"],axis=1, inplace=True)
china_epu_chinese_epu = china_epu_chinese_epu.loc["2005-01-31": "2020-06-30"]

In [31]:
ceicdata = pd.DataFrame()
ceicdata = pd.read_excel("20201101 CEIC Chinese data.xlsx")
ceicdata = ceicdata.drop(ceicdata.index[0:5])
ceicdata = ceicdata.drop(ceicdata.columns[2:4], axis=1)
ceicdata= ceicdata.drop(list(ceicdata)[-14:], axis=1)
ceicdata.rename(columns={ceicdata.columns[0]: "date"}, inplace = True)
ceicdata['date'] = pd.to_datetime(ceicdata['date']).dt.date
from pandas.tseries.offsets import MonthEnd
ceicdata['date'] = pd.to_datetime(ceicdata['date'], format="%Y-%m-%d") + MonthEnd(1)
ceicdata.set_index("date")

china_2yr = ceicdata.drop(ceicdata.columns[12:], axis=1)
china_2yr = china_2yr.drop(china_2yr.columns[1:3], axis=1)
china_2yr = china_2yr.drop(china_2yr.columns[2:], axis=1)
china_2yr = china_2yr.rename({'CN: Treasury Bond Yield: Interbank: Spot Yield: 2 year': 'china_2yr'},axis=1)
china_2yr.set_index("date", inplace=True)
china_2yr = china_2yr.loc["2005-01-31": "2020-06-30"]

china_10yr = ceicdata.drop(ceicdata.columns[12:], axis=1)
china_10yr = china_10yr.drop(china_10yr.columns[1:11], axis=1)
china_10yr = china_10yr.rename({'CN: Treasury Bond Yield: Interbank: Spot Yield: 10 year': 'china_10yr'},axis=1)
china_10yr.set_index("date", inplace=True)
china_10yr = china_10yr.loc["2005-01-31": "2020-06-30"]

china_yieldcurve_10vs2 = pd.DataFrame()
china_yieldcurve_10vs2['china_yieldcurve_2vs10'] = china_10yr['china_10yr'] - china_2yr['china_2yr']
china_yieldcurve_10vs2 = china_yieldcurve_10vs2.loc["2005-01-31": "2020-06-30"]

china_bop_monthly = ceicdata.drop(ceicdata.columns[1:30], axis=1)
china_bop_monthly = china_bop_monthly.rename({'CN: BoP: Current Account: Goods & Service': 'china_bop_monthly'},axis=1)
china_bop_monthly.set_index("date", inplace=True)
china_bop_monthly = china_bop_monthly.loc["2005-01-31": "2020-06-30"]
china_bop_monthly["china_bop_monthly"] = pd.DataFrame(china_bop_monthly['china_bop_monthly'].divide(gdp_china['gdp_china']*4)*100)

tenyr_spread = pd.DataFrame()
tenyr_spread["10yr_spread"] = china_10yr['china_10yr'].subtract(usa_10yr['usa_10yr'])

twoyr_spread = pd.DataFrame()
twoyr_spread["2yr_spread"] = china_2yr['china_2yr'].subtract(usa_2yr['usa_2yr'])


## get trade balance as a % of GDP
gdp_china = pd.DataFrame(quandl.get("NBSC/A010101_Q"))
gdp_china.columns = ["gdp_china"]
gdp_china = gdp_china.resample('M').fillna(method="ffill")
gdp_china = gdp_china.loc["2005-01-01" : "2020-06-30"]
gdp_china = gdp_china.rename_axis('date')

china_tradebalance = pd.DataFrame()
china_tradebalance = ceicdata.drop(ceicdata.columns[2:], axis=1)
china_tradebalance = china_tradebalance.rename({'CN: Trade Balance': 'china_tradebalance'},axis=1)
china_tradebalance.set_index("date", inplace=True)
china_tradebalance = china_tradebalance.loc["2005-01-31": "2020-06-30"]
china_tradebalance["china_tradebalance"] = pd.DataFrame(china_tradebalance['china_tradebalance'].divide(gdp_china['gdp_china']*4)*100)


In [32]:
fx_dataset = pd.DataFrame()
fx_dataset = china_yieldcurve_10vs2
fx_dataset = pd.merge(left=fx_dataset, right = usdcny, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_10yr, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_2yr, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_bop_monthly, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_tradebalance, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_epu_chinese_tpu, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_epu_chinese_epu, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = china_epu_english, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = usa_yieldcurve_2vs10, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = tenyr_spread, left_on='date', right_on='date')
fx_dataset = pd.merge(left=fx_dataset, right = twoyr_spread, left_on='date', right_on='date')

fx_dataset

,china_yieldcurve_2vs10,usdcny,china_10yr,china_2yr,china_bop_monthly,china_tradebalance,china_epu_chinese_tpu,china_epu_chinese_epu,china_epu_english,usa_yieldcurve_2vs10,10yr_spread,2yr_spread
date,,,,,,,,,,,,
2005-01-31,NaN,8.2765,NaN,NaN,NaN,3.50838,39,48,75.730438,0.85,NaN,NaN
2005-02-28,NaN,8.2765,NaN,NaN,NaN,2.35387,42,39,32.597446,0.77,NaN,NaN
2005-03-31,NaN,8.2765,NaN,NaN,NaN,3.57144,52,41,99.001251,0.70,NaN,NaN
2005-04-30,NaN,8.2765,NaN,NaN,NaN,2.86002,35,79,50.367886,0.55,NaN,NaN
2005-05-31,NaN,8.2765,NaN,NaN,NaN,5.59918,0,23,105.350166,0.40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29,0.4918,6.9967,2.7419,2.2501,NaN,-5.57973,132,195,646.367554,0.27,1.6119,1.3901
2020-03-31,0.6623,7.0205,2.6107,1.9484,1.67453,2.41317,105,219,734.642273,0.47,1.9107,1.7184
2020-04-30,1.2407,7.0708,2.5685,1.3278,4.91251,5.48815,193,328,819.175903,0.44,1.9285,1.1278
